# import

In [9]:
import os
import numpy as np
import math
import pandas as pd
import shutil
import re

# make gjf

# プログラムを実行

In [ ]:
maker_1=make_gjf()
maker_1.make_step2A()


1. 平行ver
2. 本ファイル
　2-A 2-B

In [7]:
class make_gjf():
    def __init__(self):
        self.path='C:\\Users\\koyama\\OSC\\interaction\\anthracene\\'
        self.df_para_step2B3=pd.read_csv(self.path+'vdw\\for_step2B+3.csv')#'C:\\Users\\youmo\\OSC\\from_tsuzuki\\Gaussian16\\vdw volume\\anthracene\\cif_vdw_6inter.csv')
        
    def make_arr(self,aa,bb,cc,A1,A2,A3,A4,A5):####i0からみた基準分子
        with open(self.path+'dimer_twist.gjf','r') as f:
                xyz=f.readlines()
        xyz[0]='%mem=15GB\n'
        xyz[1]='%nproc=20\n'
        xyz[69]='AA1      '+str(float(A1))+'\n'
        xyz[70]='AA2      '+str(float(A2))+'\n'
        xyz[71]='AA3      '+str(float(A3))+'\n'
        xyz[72]='AA4      '+str(float(90.0-A4))+'\n'##t:A4=-A2 p:A4=A2
        xyz[73]='AA5      '+str(float(-A5))+'\n'##t:A5=-A3 p:A5=A3
        xyz[74]='aa      '+str(30.0-aa)+'\n'
        xyz[75]='bb      '+str(30.0-bb)+'\n'
        xyz[76]='cc      '+str(30.0-cc)+'\n'
        xyz_add=xyz+['--Link1--\n']
        return xyz_add
    
    def make_arr_step2A(self,Rx,Ry,Rz,theta_):####i0からみた基準分子
        with open(self.path+'dimer_step2A.gjf','r') as f:
            xyz=f.readlines()
        xyz[0]='%mem=15GB\n'
        xyz[1]='%nproc=12\n'
        xyz[4]='anthracene R3={}\n'.format(Rz)
        xyz[68]='XX '+str(30-Rx)+'\n'
        xyz[69]='YY '+str(30+Ry)+'\n'
        xyz[70]='ZZ '+str(10-Rz)+'\n'
        xyz[71]='BB '+str(theta_)+'\n'
        xyz_add=xyz+['--Link1--\n']
        return xyz_add
    

    def make_step3(self):
        for A1,A2,A3,a,b,Ra,R3,theta in self.df_para:#.values:
            xyz_t= self.make_arr(a/2,b/2,0.0,A1,A2,A3,-A2,-A3)
            xyz_p= self.make_arr(0.0,b,0.0,A1,A2,A3,A2,A3)
            xyz_i0= self.make_arr(Ra,0,R3,A1,A2,A3,A2,A3)
            xyz_list=['$ RunGauss\n']+xyz_t#+xyz_p[:-1]#
            with open(self.path+'for_view\\A1={}_A2={}_A3={}_theta={}_t.gjf'.format(A1,A2,A3,theta),'w') as f:
                f.writelines(xyz_list)
            xyz_list=['$ RunGauss\n']+xyz_p#+xyz_p[:-1]#
            with open(self.path+'for_view\\A1={}_A2={}_A3={}_theta={}_p.gjf'.format(A1,A2,A3,theta),'w') as f:
                f.writelines(xyz_list)
            xyz_list=['$ RunGauss\n']+xyz_i0#+xyz_p[:-1]#
            with open(self.path+'for_view\\A1={}_A2={}_A3={}_theta={}_i0.gjf'.format(A1,A2,A3,theta),'w') as f:
                f.writelines(xyz_list)
            
    def make_step4(self):#基準分子から見た隣接層分子の位置
        cc=[]
        for idx,A1,A2,A3,a,b,Ra,R3,V,theta in self.df_para_step4[:50].values:#[155:156]
            xyz_t= self.make_arr(a/2,b/2,0.0,A1,A2,A3,-A2,-A3)
            xyz_p= self.make_arr(0.0,b,0.0,A1,A2,A3,A2,A3)
            #xyz_i0= self.make_arr(Ra,0,R3,A1,A2,A3,A2,A3)
            xyz_list=['$ RunGauss\n']+xyz_t+xyz_p[:-1]#
            with open(self.path+'step4\\6inter\\6inter_1\\A1={}_A2={}_A3={}_theta={}.inp'.format(A1,A2,A3,theta),'w') as f:
                f.writelines(xyz_list)
            cc.append('g16 < A1={}_A2={}_A3={}_theta={}.inp > A1={}_A2={}_A3={}_theta={}.log \n'.format(A1,A2,A3,theta,A1,A2,A3,theta))
        with open(self.path+'step4\\6inter\\6inter_1\\cc01.txt','w') as f:
            f.writelines(cc)

    def make_step2B3(self):
        #RaRbをDFT探索
        cc=[]
        for A1,A2,A3,a,b,Ra,R3,V in self.df_para_step2B3.values:#[155:156]
            A2=int(A2)
            xyz_t= self.make_arr(a/2,b/2,0.0,A1,A2,A3,-A2,-A3)
            xyz_p= self.make_arr(0.0,b,0.0,A1,A2,A3,A2,A3)
            xyz_i0= self.make_arr(Ra+a,0,R3,A1,A2,A3,A2,A3)
            xyz_ip1= self.make_arr(Ra+a,b,R3,A1,A2,A3,A2,A3)#i0+p
            xyz_it1= self.make_arr(Ra+3*a/2,b/2,R3,A1,A2,A3,-A2,-A3)#i0+t
            xyz_ip2= self.make_arr(Ra+a,-b,R3,A1,A2,A3,A2,A3)#i0-p
            xyz_it3= self.make_arr(Ra+a/2,-b/2,R3,A1,A2,A3,-A2,-A3)#i0-t
            xyz_list=['$ RunGauss\n']+xyz_t+xyz_p+xyz_i0+xyz_ip1+xyz_it1+xyz_ip2+xyz_it3[:-1]#+xyz_p[:-1]#
            with open(self.path+'step2B3\\A2={}_a={}_b={}.inp'.format(A2,a,b),'w') as f:
                f.writelines(xyz_list)
            
    def divide_files(self,sep=60):#一フォルダのファイル数
        path_inp=self.path+'step2B3\\'

        int_folder=0
        file_list_inp=[file for file in os.listdir(path_inp) if file[-4:]=='.inp']
        while len(file_list_inp)!=0:
            
            #フォルダ増やす
            int_folder+=1
            path_inpn=path_inp+'inp'+str(int_folder)
            os.mkdir(path_inpn)
            
            #fileをsepずつ新フォルダに移す
            for file in file_list_inp[:min(len(file_list_inp),sep)]:
                shutil.move(path_inp+file,path_inpn)
            file_list_inp=[file for file in os.listdir(path_inp) if file[-4:]=='.inp']

    
    def make_exe(self):
        path_inp=self.path+'step2B3\\'
        #mkdir
        cc_list_1=['#!/bin/sh \n',
                    '#$ -S /bin/sh \n',
                    '#$ -cwd \n',
                    '#$ -V \n',
                    '#$ -q all.q \n',
                    '#$ -pe OpenMP 40 \n',
                    '\n',
                    'hostname \n',
                    '\n',
                    'export g16root=/home/g03 \n',
                    'source $g16root/g16/bsd/g16.profile \n',
                    '\n',
                    'export GAUSS_SCRDIR=/home/scr/$JOB_ID \n',
                    'mkdir /home/scr/$JOB_ID \n',
                    '\n']
        cc_list=[]
        cc_list_2=['\n',
                  'rm -rf /home/scr/$JOB_ID \n',
                  '\n',
                  '\n',
                  '#sleep 500 \n'
                  ]

        file_list_inp=[fold for fold in os.listdir(path_inp) if fold[:3]=='inp']
        N_folder=len(file_list_inp)
        for n_folder in range(1,N_folder+1):
            path_inpn=self.path+'step2B3\\inp'+str(n_folder)
            file_list=[file[:-4] for file in os.listdir(path_inpn) if file[-4:]=='.inp']
            cc_list=[]
            for file_name in file_list:
                line='g16 < '+file_name+'.inp > '+file_name+'.log \n'
                cc_list.append(line)
            with open(path_inpn+'\\n02.r1','w') as f:
                f.writelines(cc_list_1+cc_list+cc_list_2)

##############
#             xyz_i0= self.make_arr(Ra+a,0,R3,A1,A2,A3,A2,A3)#i0=a+c
#             xyz_ip1= self.make_arr(Ra+a,b,R3,A1,A2,A3,A2,A3)#i0+p
#             xyz_it1= self.make_arr(Ra+3*a/2,b/2,R3,A1,A2,A3,-A2,-A3)#i0+t
#             xyz_it2= self.make_arr(Ra+3*a/2,-b/2,R3,A1,A2,A3,-A2,-A3)#i0+t-p
#             xyz_ip2= self.make_arr(Ra+a,-b,R3,A1,A2,A3,A2,A3)#i0-p
#             xyz_it3= self.make_arr(Ra+a/2,-b/2,R3,A1,A2,A3,-A2,-A3)#i0-t
#             xyz_it4= self.make_arr(Ra+a/2,b/2,R3,A1,A2,A3,-A2,-A3)#i0-t+p
#             xyz_list=['$ RunGauss\n']+xyz_i0+xyz_ip1+xyz_it1+xyz_it2+xyz_ip2+xyz_it3+xyz_it4[:-1]#+xyz_p[:-1]#
#             with open(self.path+'step4\\step4_4\\A1={}_A2={}_A3={}_theta={}.inp'.format(A1,A2,A3,theta),'w') as f:
#                 f.writelines(xyz_list)
#             cc.append('g16 < A1={}_A2={}_A3={}_theta={}.inp > A1={}_A2={}_A3={}_theta={}.log \n'.format(A1,A2,A3,theta,A1,A2,A3,theta))
#         with open(self.path+'step4\\step4_4\\cc01.txt','w') as f:
#             f.writelines(cc)
########################################    

In [11]:
maker_1=make_gjf()#para:(R1=4.6,R2=1.0,heri=53.0,R3t=2.3)
#maker_1.make_step2B3()
maker_1.divide_files(sep=28)
maker_1.make_exe()